In [1]:
# === STEP 1: Install dependencies ===
# pip install pandas openpyxl scikit-learn

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
import numpy as np

# === STEP 2: Load and Prepare the Training Data (labeled) ===
df_train = pd.read_excel("test01.xlsx")

# Combine system, description, and equipment into one text column
df_train["text"] = (
    df_train["Systeme"].astype(str) + " | " +
    df_train["Description"].astype(str) + " | " +
    df_train["Description de l'équipement"].astype(str)
)

# Drop any rows with missing labels
df_train = df_train.dropna(subset=["Fiabilité Intégrité", "Disponibilté", "Process Safety"])

# Define features (X) and labels (y)
X_train = df_train["text"]
y_train = df_train[["Fiabilité Intégrité", "Disponibilté", "Process Safety"]]

# === STEP 3: Train the Multi-Output Regression Model ===
model = Pipeline([
    ("tfidf", TfidfVectorizer(max_features=1000, ngram_range=(1, 2))),
    ("regressor", MultiOutputRegressor(Ridge()))
])

model.fit(X_train, y_train)


,steps,"[('tfidf', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [2]:

# === STEP 4: Load New Anomalies to Score ===
df_new = pd.read_excel("test01wfi.xlsx")

# Combine same columns as above
df_new["text"] = (
    df_new["Systeme"].astype(str) + " | " +
    df_new["Description"].astype(str) + " | " +
    df_new["Description de l'équipement"].astype(str)
)

# === STEP 5: Predict Scores ===
X_new = df_new["text"]
predictions = model.predict(X_new)

# Round and clip scores between 1 and 5
predicted_scores = pd.DataFrame(predictions, columns=["Fiabilité Intégrité", "Disponibilté", "Process Safety"])
predicted_scores = predicted_scores.round().clip(1, 5).astype(int)

# === STEP 6: Combine and Export Results ===
df_result = pd.concat([df_new, predicted_scores], axis=1)
display(df_result.head(20))
df_result.to_excel("scored_anomalies.xlsx", index=False)

print("✅ Scoring complete! File saved as 'scored_anomalies.xlsx'")


,Num_equipement,Systeme,Description,Description de l'équipement,text,Fiabilité Intégrité,Disponibilté,Process Safety
0,3f68d058-b105-46f0-b137-6ce7695b85d1,e4f67f5f-be46-4ce0-ad14-68575d3cf0cd,Soufflage avec de l'air des radiateur de refro...,TRANSFO PRINCIPAL,e4f67f5f-be46-4ce0-ad14-68575d3cf0cd | Souffla...,2,5,1
1,3f68d058-b105-46f0-b137-6ce7695b85d1,e4f67f5f-be46-4ce0-ad14-68575d3cf0cd,Bruit moto-ventilateur transformateur principa...,TRANSFO PRINCIPAL,e4f67f5f-be46-4ce0-ad14-68575d3cf0cd | Bruit m...,4,5,2
2,3f68d058-b105-46f0-b137-6ce7695b85d1,e4f67f5f-be46-4ce0-ad14-68575d3cf0cd,manque éclairage transformateur principal,TRANSFO PRINCIPAL,e4f67f5f-be46-4ce0-ad14-68575d3cf0cd | manque ...,3,4,4
3,3f68d058-b105-46f0-b137-6ce7695b85d1,e4f67f5f-be46-4ce0-ad14-68575d3cf0cd,Arrêt 7 jours U3 AVR 2021: Entretien préventif...,TRANSFO PRINCIPAL,e4f67f5f-be46-4ce0-ad14-68575d3cf0cd | Arrêt 7...,2,5,1
4,3f68d058-b105-46f0-b137-6ce7695b85d1,e4f67f5f-be46-4ce0-ad14-68575d3cf0cd,Dégagement des portes grillage haut suite risq...,TRANSFO PRINCIPAL,e4f67f5f-be46-4ce0-ad14-68575d3cf0cd | Dégagem...,2,4,2
5,3f68d058-b105-46f0-b137-6ce7695b85d1,e4f67f5f-be46-4ce0-ad14-68575d3cf0cd,prévoir changer le silicagel du transformateur...,TRANSFO PRINCIPAL,e4f67f5f-be46-4ce0-ad14-68575d3cf0cd | prévoir...,3,5,2
6,3f68d058-b105-46f0-b137-6ce7695b85d1,e4f67f5f-be46-4ce0-ad14-68575d3cf0cd,netoyage et souflage des radiateurs du transfo...,TRANSFO PRINCIPAL,e4f67f5f-be46-4ce0-ad14-68575d3cf0cd | netoyag...,2,4,2
7,3f68d058-b105-46f0-b137-6ce7695b85d1,e4f67f5f-be46-4ce0-ad14-68575d3cf0cd,Prévoir la remise en état porte transformateur...,TRANSFO PRINCIPAL,e4f67f5f-be46-4ce0-ad14-68575d3cf0cd | Prévoir...,3,4,2
8,3f68d058-b105-46f0-b137-6ce7695b85d1,e4f67f5f-be46-4ce0-ad14-68575d3cf0cd,Présence trace d'huile au niveau sortie borne ...,TRANSFO PRINCIPAL,e4f67f5f-be46-4ce0-ad14-68575d3cf0cd | Présenc...,3,4,2
9,3f68d058-b105-46f0-b137-6ce7695b85d1,e4f67f5f-be46-4ce0-ad14-68575d3cf0cd,prévoir le contrôle des indicateur température...,TRANSFO PRINCIPAL,e4f67f5f-be46-4ce0-ad14-68575d3cf0cd | prévoir...,2,3,1


✅ Scoring complete! File saved as 'scored_anomalies.xlsx'
